In [1]:
import torch

In [28]:
x=torch.ones(2,2, requires_grad=True)
print(x)
y=x+2
print(y)
z= y * y 
print(z)
out = z.mean()
print(out)
#backprop
out.backward()
#out = (((x1+2)^2) + ... + ((x4+2)^2))/4
#dout/dx = 2(x1+2) /4 + ... + ... +
#x.grad = [1.5,1.5,1.5,1.5]
print(x.grad)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
tensor([[9., 9.],
        [9., 9.]], grad_fn=<MulBackward0>)
tensor(9., grad_fn=<MeanBackward0>)
tensor([[1.5000, 1.5000],
        [1.5000, 1.5000]])


In [33]:
t = torch.tensor([[1, 2], [5, 7]], dtype=float, requires_grad=True)
t

tensor([[1., 2.],
        [5., 7.]], dtype=torch.float64, requires_grad=True)

In [34]:
l = t.sum()
l

tensor(15., dtype=torch.float64, grad_fn=<SumBackward0>)

In [35]:
l = 3*l
l

tensor(45., dtype=torch.float64, grad_fn=<MulBackward0>)

In [36]:
l.backward()

tensor(45., dtype=torch.float64, grad_fn=<MulBackward0>)

In [37]:
t.grad

tensor([[3., 3.],
        [3., 3.]], dtype=torch.float64)

In [19]:
x = torch.ones(2, 2, requires_grad=True)
print(x)
y = x + 2 
print(y) 
print(y.grad_fn) 
print()
z = y * y * 3
out = z.mean()
print(z)
print(out)
out.backward() # print gradients d(out)/dx
print(x.grad) # tensor([[4.5000, 4.5000], [4.5000, 4.5000]])

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [38]:
class OneLayerNet(torch.nn.Module):
  def __init__(self, num_params, num_classes):
    super().__init__()
    self.linear = torch.nn.Linear(num_params, num_classes)
  
  def forward(self, x):
    # x = (batch, num_params)
    # output = (batch, num_classes)
    return self.linear(x)

net = OneLayerNet(28*28, 10)
print(net)

OneLayerNet(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)


In [39]:
x = torch.rand(28*28)
print(x.shape)
x = x.unsqueeze(0)
print(x.shape)
out = net(x)
print(out.shape)

torch.Size([784])
torch.Size([1, 784])
torch.Size([1, 10])


In [40]:
# out = w (784) * x (784x10) + b (10)
[p.shape for p in net.parameters()]

[torch.Size([10, 784]), torch.Size([10])]

In [41]:
from torch.functional import F


loss_fn = torch.nn.MSELoss()
target = torch.zeros(1, 10)
target[0, 0] = 1
print(target.shape)
# l = F.mse_loss(out, target)
l = loss_fn(out, target)
print(l)

torch.Size([1, 10])
tensor(0.3406, grad_fn=<MseLossBackward0>)


In [42]:
net.zero_grad()
print([p.grad for p in net.parameters()])
l.backward()
print([p.grad for p in net.parameters()])

[None, None]
[tensor([[-5.8406e-02, -2.1710e-01, -9.9268e-02,  ..., -1.2211e-01,
         -1.2161e-01, -2.5773e-02],
        [ 1.5052e-02,  5.5949e-02,  2.5583e-02,  ...,  3.1469e-02,
          3.1341e-02,  6.6419e-03],
        [ 2.8428e-02,  1.0567e-01,  4.8316e-02,  ...,  5.9433e-02,
          5.9192e-02,  1.2544e-02],
        ...,
        [-8.2097e-03, -3.0516e-02, -1.3953e-02,  ..., -1.7164e-02,
         -1.7094e-02, -3.6227e-03],
        [ 2.1253e-04,  7.8999e-04,  3.6122e-04,  ...,  4.4433e-04,
          4.4253e-04,  9.3782e-05],
        [-1.7620e-02, -6.5495e-02, -2.9947e-02,  ..., -3.6838e-02,
         -3.6689e-02, -7.7751e-03]]), tensor([-0.2767,  0.0713,  0.1347, -0.0669,  0.0784, -0.0773,  0.1067, -0.0389,
         0.0010, -0.0835])]


In [43]:
# criterion = loss_fn
loss_fn = torch.nn.CrossEntropyLoss()
target = torch.tensor([3]).long()
print(target.shape)
x = torch.rand(28*28)
x = x.unsqueeze(0)
out = net(x)
print(out.shape)
l = loss_fn(out, target)
print(l)
l.backward()

torch.Size([1])
torch.Size([1, 10])
tensor(2.4756, grad_fn=<NllLossBackward0>)


In [44]:
print([p.grad for p in net.parameters()])
learning_rate = 0.01
# for f in net.parameters():
#   f.data -= f.grad.data * learning_rate

opt = torch.optim.SGD(net.parameters(), learning_rate)
opt.step()
opt.zero_grad()
print([p.grad for p in net.parameters()])

[tensor([[-0.0056, -0.2136, -0.0925,  ..., -0.1109, -0.1102, -0.0004],
        [ 0.1752,  0.0666,  0.0462,  ...,  0.0655,  0.0661,  0.0837],
        [ 0.1206,  0.1118,  0.0602,  ...,  0.0790,  0.0792,  0.0569],
        ...,
        [ 0.0548, -0.0263, -0.0058,  ..., -0.0038, -0.0034,  0.0267],
        [ 0.0571,  0.0046,  0.0077,  ...,  0.0126,  0.0128,  0.0275],
        [ 0.0527, -0.0608, -0.0209,  ..., -0.0219, -0.0214,  0.0261]]), tensor([-0.2177,  0.2502,  0.2377, -0.9828,  0.2171,  0.0120,  0.2411,  0.0315,
         0.0646, -0.0049])]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]


In [45]:
import torchvision

transform = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize(0.5, 0.5)
                  ])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
      download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False,
      download=True, transform=transform)



In [46]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
    shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
    shuffle=False, num_workers=4)

In [49]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')